In [1]:
%run connect_to_api.ipynb
%run load_dependencies.ipynb
%run misc_functions.ipynb

In [2]:
def peak_data_report(map_ids, from_date = None, to_date = datetime.now(),num_days = 7,features = ['hashtags','post_url'],limit = None,sheet = False):
    if limit == None:
        limit = ''
    else:
        limit = '&limit='+str(limit)
    if type(map_ids) != list:
        map_ids = [map_ids]
    to_write = {}
    
    for map_id in map_ids:
        print(f'...Fetching peak objects for map {map_id}')
        peak_items_df = pd.DataFrame()
        groups = sb.Groups(map_id)
        group_data = {}
        from_date = to_date - timedelta(days = num_days)

        for g in groups:
            group_data[g['group_no']] = g['name']

        for f in features:
            for group in group_data:
                url = (f"https://api.graphika.com/focus/{map_id}/group/{group}/{f}?orderBy=engagement_count&orderDirection=-1&format=json{limit}&offset=0")
                r = re.get(url,auth=(username, pswd))
                peak_items = r.json()
                df = pd.DataFrame([n for n in peak_items["results"]])
                df['map_id'] = map_id
                df.peak_date = pd.to_datetime(df['peak_date'])
                df_peak = df[df["peak_date"]>=from_date]
                df_peak = df_peak[df_peak["peak_date"]<to_date]
                df_peak["hit_type"] = f
                df_peak["group"] = group_data[group]

                peak_items_df = pd.concat([peak_items_df,df_peak])
        peak_items_df['starship'] = peak_items_df.apply(get_starship_links,axis = 1)
        cols = ["group",
            "hit_type",
            "engagement_count",
            "peak_date",
            "target",
            "starship",
            "focus",
            "total_engagement_count",
            "unique_source_count",
            "peakedness",
            "peak_hits_count",
            "map_id"]

        if "mentions" in features:
            peak_items_df.loc[peak_items_df.feature_type == "mentions","target"] = \
            peak_items_df.loc[peak_items_df.feature_type == "mentions","target"]\
            .apply(lambda x:f'=HYPERLINK("http://www.twitter.com/i/user/{x}", "{x}")')

            cols.extend["username",
                "name",
                "profile_image_url",
                "num_following",
                "num_followers",
                "account_url"]
                #"retweeted_text"]

        if "retweets" in features:
            peak_items_df.loc[peak_items_df.hit_type == "retweets","username"] = peak_items_df[peak_items_df.hit_type == "retweets"]["retweeted_username"] 
            peak_items_df.loc[peak_items_df.hit_type == "retweets","name"] = peak_items_df[peak_items_df.hit_type == "retweets"]["retweeted_user_name"] 

            peak_items_df.loc[peak_items_df.feature_type == "retweets","target"] = peak_items_df.loc[peak_items_df.feature_type == "retweets","target"]\
            .apply(lambda x:f'=HYPERLINK("http://www.twitter.com/x/status/{x}", "{x}")')

            cols.append("retweeted_text")
        to_write[str(map_id)] = peak_items_df[cols]
    
    if sheet:
#         write_to_sheet(to_write,"Peak Objects Report {}".format(str(datetime.now())[:10]))
        write_to_sheet(to_write,'Peak Objects Report [Domestic Narrative Monitoring]')
    else:
        for k in to_write:
            writer = pd.ExcelWriter(f'/Users/avneeshchandra/Downloads/peak_objects_report - {str(to_date)[:10]}.xlsx')
            to_write[k].to_excel(writer,str(k),index = False)
            writer.save()

In [3]:
peak_data_report([2736,2364,2289,2690,2712],num_days = 7,limit = 100,sheet = True)

...Fetching peak objects for map 2736
...Fetching peak objects for map 2364
...Fetching peak objects for map 2289
...Fetching peak objects for map 2690
...Fetching peak objects for map 2712
...Opening Sheet
...Writing 2736
...Writing 2364
...Writing 2289
...Writing 2690


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


...Writing 2712
...Sheet now has:
<Worksheet '2712' id:2004107082> (191, 12)
<Worksheet '2736' id:306893847> (241, 12)
<Worksheet '2364' id:140606886> (152, 12)
<Worksheet '2289' id:818105275> (375, 12)
<Worksheet '2690' id:372434718> (292, 12)
...Sheet available at: https://docs.google.com/spreadsheets/d/1ifb4NipdDw6oeztIbyzgIPkemGm6MbW-7_HMorTPOaQ
